In [6]:
%matplotlib inline

import os
import numpy as np
import pandas as pd
from glob import glob
import nibabel as nib
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


from PIL import Image
import matplotlib.cm as cm
import cv2

from sklearn.preprocessing import MinMaxScaler

In [7]:
print(np.__version__)
print(pd.__version__)

print(nib.__version__)
print(cv2.__version__)

1.18.5
1.2.2
3.2.2
4.5.1


In [33]:
data_dir= '../' #put your data path

In [34]:
sev_dir = os.path.join(data_dir, 'severance')

In [35]:
asan_dir = os.path.join(data_dir, 'asan')

# 1. Check Data

## 1.1. Get pt ID list

In [36]:
# get pt id list
sev_gbm_pt_list = os.listdir(os.path.join(sev_dir, 'GBM'))
sev_meta_pt_list = os.listdir(os.path.join(sev_dir, 'meta'))

asan_gbm_pt_list = os.listdir(os.path.join(asan_dir, 'GBM'))
asan_meta_pt_list = os.listdir(os.path.join(asan_dir, 'meta'))

In [37]:
print('Sev_GBM : ', len(sev_gbm_pt_list))
print('Sev_Metastasis : ', len(sev_meta_pt_list))
print('Asan_GBM : ', len(asan_gbm_pt_list))
print('Asan_Metastasis : ', len(asan_meta_pt_list))

Sev_GBM :  308
Sev_Metastasis :  171
Asan_GBM :  101
Asan_Metastasis :  42


In [38]:
# check overlap
print('Sev_GBM : ', len(set(sev_gbm_pt_list)))
print('Sev_Metastasis : ', len(set(sev_meta_pt_list)))
print('Asan_GBM : ', len(set(asan_gbm_pt_list)))
print('Asan_Metastasis : ', len(set(asan_meta_pt_list)))

Sev_GBM :  308
Sev_Metastasis :  171
Asan_GBM :  101
Asan_Metastasis :  42


In [39]:
# remove weird data pt
# Sev - GBM : [26, 33, 52, 114, 122, 132, 145, 156]
# Sev - meta : [6, 43]

sev_gbm_pt_list = np.delete(np.array(sev_gbm_pt_list), [26, 33, 52, 114, 122, 132, 145, 156])
sev_meta_pt_list = np.delete(np.array(sev_meta_pt_list), [6, 43])

In [40]:
# final data amount to be used
print('Sev_GBM : ', len(set(sev_gbm_pt_list)))
print('Sev_Metastasis : ', len(set(sev_meta_pt_list)))
print('Asan_GBM : ', len(set(asan_gbm_pt_list)))
print('Asan_Metastasis : ', len(set(asan_meta_pt_list)))

Sev_GBM :  300
Sev_Metastasis :  169
Asan_GBM :  101
Asan_Metastasis :  42


In [45]:
# making pt id list in csv file 
d = dict( Sev_GBM = sev_gbm_pt_list, 
         Sev_Meta = sev_meta_pt_list,
        Asan_GBM = np.array(asan_gbm_pt_list), 
        Asan_Meta = np.array(asan_meta_pt_list))
    
df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in d.items() ]))

df.to_csv('Brain-GBMmeta_pt_list.csv')

## 1.2. Get all data paths

In [41]:
# load segmentation data (a.k.a mask)
sev_gbm_dir_mask = glob(os.path.join(sev_dir, 'GBM', '*/new_segmentation.nii.gz'))
sev_meta_dir_mask = glob(os.path.join(sev_dir, 'meta', '*/new_segmentation.nii.gz'))
asan_gbm_dir_mask = glob(os.path.join(asan_dir, 'GBM', '*/new_segmentation.nii.gz'))
asan_meta_dir_mask = glob(os.path.join(asan_dir, 'meta', '*/new_segmentation.nii.gz'))

In [42]:
# Load CT1 data
sev_gbm_dir_ct1 = glob(os.path.join(sev_dir, 'GBM', '*/CT1_r2s_bet_reg_znormed.nii.gz'))
sev_meta_dir_ct1 = glob(os.path.join(sev_dir, 'meta', '*/CT1_r2s_bet_reg_znormed.nii.gz'))
asan_gbm_dir_ct1 = glob(os.path.join(asan_dir, 'GBM', '*/CT1_r2s_bet_reg_znormed.nii.gz'))
asan_meta_dir_ct1 = glob(os.path.join(asan_dir, 'meta', '*/CT1_r2s_bet_reg_znormed.nii.gz'))

In [43]:
# Load T2 data
sev_gbm_dir_t2 = glob(os.path.join(sev_dir, 'GBM', '*/T2_r2s_bet_reg_znormed.nii.gz'))
sev_meta_dir_t2 = glob(os.path.join(sev_dir, 'meta', '*/T2_r2s_bet_reg_znormed.nii.gz'))
asan_gbm_dir_t2 = glob(os.path.join(asan_dir, 'GBM', '*/T2_r2s_bet_reg_znormed.nii.gz'))
asan_meta_dir_t2 = glob(os.path.join(asan_dir, 'meta', '*/T2_r2s_bet_reg_znormed.nii.gz'))

In [44]:
# remove weird data - sev_gbm: [26, 33, 52, 114, 122, 132, 145, 156]; sev_meta: [6, 43]
sev_gbm_dir_mask = np.delete(np.array(sev_gbm_dir_mask), [26, 33, 52, 114, 122, 132, 145, 156])
sev_gbm_dir_ct1 = np.delete(np.array(sev_gbm_dir_ct1), [26, 33, 52, 114, 122, 132, 145, 156])
sev_gbm_dir_t2 = np.delete(np.array(sev_gbm_dir_t2), [26, 33, 52, 114, 122, 132, 145, 156])

sev_meta_dir_mask = np.delete(np.array(sev_meta_dir_mask), [6, 43])
sev_meta_dir_ct1 = np.delete(np.array(sev_meta_dir_ct1), [6, 43])
sev_meta_dir_t2 = np.delete(np.array(sev_meta_dir_t2), [6, 43])

## 1.3. Check shapes

In [8]:
# check if there's shape difference btw modalities
def check_shape_difference(mask, ct1, t2): # put dir list (*nii.gz)
    for i in tqdm(range(len(mask))):
        mask_array = nib.load(mask[i]).get_fdata()
        ct1_array = nib.load(ct1[i]).get_fdata()
        t2_array = nib.load(t2[i]).get_fdata()
        
        if mask_array.shape != ct1_array.shape:
            print(mask[i])
        if mask_array.shape != t2_array.shape:
            print(mask[i])

In [48]:
check_shape_difference(sev_gbm_dir_mask, sev_gbm_dir_ct1, sev_gbm_dir_t2)
check_shape_difference(sev_meta_dir_mask, sev_meta_dir_ct1, sev_meta_dir_t2)

check_shape_difference(asan_gbm_dir_mask, asan_gbm_dir_ct1, asan_gbm_dir_t2)
check_shape_difference(asan_meta_dir_mask, asan_meta_dir_ct1, asan_meta_dir_t2) 

#no shape diffence!

# 2. Extracting top 5 slices by tumor segmentation data

## 2.1. Get top 5 tumor lesion slice index

In [9]:
# count non-zero values in each layer of a single mask(segmentation) file and get top 5 layer indexes
def get_slice_index(mri_array):
    mri_shape = mri_array.shape
    
    non_zero=[]
    for i in range(mri_shape[2]):
        data, count = np.unique(mri_array[:,:,i], return_counts=True)
        non_zero_pixel = np.sum(count[np.where(data>0)])
        non_zero.append(non_zero_pixel)
    
    # sorting index with no_zero pixel amount 
    non_zero = sorted(range(len(non_zero)), key=lambda i: non_zero[i])[-5:]
    
    return non_zero

In [10]:
def total_index(dir):
    top_5_index=[]
    for mask_path in tqdm(dir):
        mri_array = nib.load(mask_path).get_fdata()
        non_zero = get_slice_index(mri_array)
        top_5_index.append(non_zero)
    return top_5_index

In [ ]:
sev_gbm_index = total_index(sev_gbm_dir_mask)
sev_meta_index = total_index(sev_meta_dir_mask)
asan_gbm_index = total_index(asan_gbm_dir_mask)
asan_meta_index = total_index(asan_meta_dir_mask)

## 2.2. Extracting top 5 slices: using INDEX in 2.1.

Severance : Internal data  
Asan : External Test Data

In [11]:
def get_top5slices(path_list, top_5_index):
    data = []
    
    for idx, mri in enumerate(tqdm(path_list)):
        mri_array = nib.load(mri).get_fdata()        
        mri_array_ = mri_array[:, :, top_5_index[idx]]
        data.append(mri_array_) 

    return data

### 2.2.1. Internal data

In [ ]:
# mask
int_mask_gbm = get_top5slices(sev_gbm_dir_mask, sev_gbm_index)
int_mask_meta = get_top5slices(sev_meta_dir_mask, sev_meta_index)

In [ ]:
# ct1
int_ct1_gbm = get_top5slices(sev_gbm_dir_ct1, sev_gbm_index)
int_ct1_meta = get_top5slices(sev_meta_dir_ct1, sev_meta_index)

In [ ]:
# t2 
int_t2_gbm = get_top5slices(sev_gbm_dir_t2, sev_gbm_index)
int_t2_meta = get_top5slices(sev_meta_dir_t2, sev_meta_index)

In [68]:
# check data amount
print('int_mask_gbm : ', len(int_mask_gbm))
print('int_mask_meta : ', len(int_mask_meta))
print('int_ct1_gbm : ', len(int_ct1_gbm))
print('int_ct1_meta : ', len(int_ct1_meta))
print('int_t2_gbm : ', len(int_t2_gbm))
print('int_t2_meta : ', len(int_t2_meta))

int_mask_gbm :  300
int_mask_meta :  169
int_ct1_gbm :  300
int_ct1_meta :  169
int_t2_gbm :  300
int_t2_meta :  169


In [ ]:
# visualizing extracted data
n=0
f, axarr = plt.subplots(10, 10, sharex=True, sharey=True, figsize=(20, 20))
for i in range(10):
    for j in range(10):
        # put dataset you want to see
        axarr[i, j].imshow(int_mask_gbm[n+10*i+j][:,:,4], cmap='gray')
        axarr[i, j].axis('off')
        axarr[i, j].set_title(n+10*i+j)
plt.show()

### 2.2.2. External test data

In [ ]:
# mask
ext_mask_gbm = get_top5slices(asan_gbm_dir_mask, asan_gbm_index)
ext_mask_meta = get_top5slices(asan_meta_dir_mask, asan_meta_index)

In [ ]:
# ct1 
ext_ct1_gbm = get_top5slices(asan_gbm_dir_ct1, asan_gbm_index)
ext_ct1_meta = get_top5slices(asan_meta_dir_ct1, asan_meta_index)

In [ ]:
# t2
ext_t2_gbm = get_top5slices(asan_gbm_dir_t2, asan_gbm_index)
ext_t2_meta = get_top5slices(asan_meta_dir_t2, asan_meta_index)

In [77]:
# check data amount
print('ext_mask_gbm : ', len(ext_mask_gbm))
print('ext_mask_meta : ', len(ext_mask_meta))
print('ext_ct1_gbm : ', len(ext_ct1_gbm))
print('ext_ct1_meta : ', len(ext_ct1_meta))
print('ext_t2_gbm : ', len(ext_t2_gbm))
print('ext_t2_meta : ', len(ext_t2_meta))

ext_mask_gbm :  101
ext_mask_meta :  42
ext_ct1_gbm :  101
ext_ct1_meta :  42
ext_t2_gbm :  101
ext_t2_meta :  42


In [ ]:
# visualizing extracted data
n=0
f, axarr = plt.subplots(10, 10, sharex=True, sharey=True, figsize=(20, 20))
for i in range(10):
    for j in range(10):
        # put dataset you want to see
        axarr[i, j].imshow(ext_mask_gbm[n+10*i+j][:,:,4], cmap='gray')
        axarr[i, j].axis('off')
        axarr[i, j].set_title(n+10*i+j)
plt.show()

# 3. Data Preprocessing


## 3.1. Internal data train, validation, test split
make sure single pt's data (five in this case) go into single group

In [145]:
gbm_train_idx, gbm_test_idx = train_test_split(np.arange(0, len(int_mask_gbm)), test_size=0.15, random_state=42)
gbm_train_idx, gbm_val_idx = train_test_split(np.arange(0, len(gbm_train_idx)), test_size=0.1, random_state=42)

meta_train_idx, meta_test_idx = train_test_split(np.arange(0, len(int_mask_meta)), test_size=0.15, random_state=42)
meta_train_idx, meta_val_idx = train_test_split(np.arange(0, len(meta_train_idx)), test_size=0.1, random_state=42)

In [146]:
x_train_gbm_mask = np.array(int_mask_gbm)[gbm_train_idx]
x_train_gbm_ct1 = np.array(int_ct1_gbm)[gbm_train_idx]
x_train_gbm_t2 = np.array(int_t2_gbm)[gbm_train_idx]

x_val_gbm_mask = np.array(int_mask_gbm)[gbm_val_idx]
x_val_gbm_ct1 = np.array(int_ct1_gbm)[gbm_val_idx]
x_val_gbm_t2 = np.array(int_t2_gbm)[gbm_val_idx]

x_test_gbm_mask = np.array(int_mask_gbm)[gbm_test_idx]
x_test_gbm_ct1 = np.array(int_ct1_gbm)[gbm_test_idx]
x_test_gbm_t2 = np.array(int_t2_gbm)[gbm_test_idx]

In [147]:
x_train_meta_mask = np.array(int_mask_meta)[meta_train_idx]
x_train_meta_ct1 = np.array(int_ct1_meta)[meta_train_idx]
x_train_meta_t2 = np.array(int_t2_meta)[meta_train_idx]

x_val_meta_mask = np.array(int_mask_meta)[meta_val_idx]
x_val_meta_ct1 = np.array(int_ct1_meta)[meta_val_idx]
x_val_meta_t2 = np.array(int_t2_meta)[meta_val_idx]

x_test_meta_mask = np.array(int_mask_meta)[meta_test_idx]
x_test_meta_ct1 = np.array(int_ct1_meta)[meta_test_idx]
x_test_meta_t2 = np.array(int_t2_meta)[meta_test_idx]

In [148]:
len(x_train_gbm_mask), len(x_val_gbm_mask), len(x_test_gbm_mask), len(x_train_meta_mask), len(x_val_meta_mask), len(x_test_meta_mask)

(229, 26, 45, 128, 15, 26)

In [133]:
len(x_train_gbm_ct1), len(x_val_gbm_ct1), len(x_test_gbm_ct1), len(x_train_meta_ct1), len(x_val_meta_ct1), len(x_test_meta_ct1)

(229, 26, 45, 128, 15, 26)

In [134]:
len(x_train_gbm_t2), len(x_val_gbm_t2), len(x_test_gbm_t2), len(x_train_meta_t2), len(x_val_meta_t2), len(x_test_meta_t2)

(229, 26, 45, 128, 15, 26)

In [31]:
# turning 3d array into 2d array
def TurnInto_2DArray(fiveslices_list): #x_train_gbm, x_train_meta etc. 
    twod_data = []
    for threed_array in fiveslices_list:
        for i in range(5):
            twod_data.append(threed_array[:,:,i])
    return twod_data

In [153]:
x_train_gbm_mask = TurnInto_2DArray(x_train_gbm_mask)
x_train_gbm_ct1 = TurnInto_2DArray(x_train_gbm_ct1)
x_train_gbm_t2 = TurnInto_2DArray(x_train_gbm_t2)

x_val_gbm_mask = TurnInto_2DArray(x_val_gbm_mask)
x_val_gbm_ct1 = TurnInto_2DArray(x_val_gbm_ct1)
x_val_gbm_t2 = TurnInto_2DArray(x_val_gbm_t2)

x_test_gbm_mask = TurnInto_2DArray(x_test_gbm_mask)
x_test_gbm_ct1 = TurnInto_2DArray(x_test_gbm_ct1)
x_test_gbm_t2 = TurnInto_2DArray(x_test_gbm_t2)

x_ext_gbm_mask = TurnInto_2DArray(ext_mask_gbm)
x_ext_gbm_ct1 = TurnInto_2DArray(ext_ct1_gbm)
x_ext_gbm_t2 = TurnInto_2DArray(ext_t2_gbm)

In [154]:
x_train_meta_mask = TurnInto_2DArray(x_train_meta_mask)
x_train_meta_ct1 = TurnInto_2DArray(x_train_meta_ct1)
x_train_meta_t2 = TurnInto_2DArray(x_train_meta_t2)

x_val_meta_mask = TurnInto_2DArray(x_val_meta_mask)
x_val_meta_ct1 = TurnInto_2DArray(x_val_meta_ct1)
x_val_meta_t2 = TurnInto_2DArray(x_val_meta_t2)

x_test_meta_mask = TurnInto_2DArray(x_test_meta_mask)
x_test_meta_ct1 = TurnInto_2DArray(x_test_meta_ct1)
x_test_meta_t2 = TurnInto_2DArray(x_test_meta_t2)

x_ext_meta_mask = TurnInto_2DArray(ext_mask_meta)
x_ext_meta_ct1 = TurnInto_2DArray(ext_ct1_meta)
x_ext_meta_t2 = TurnInto_2DArray(ext_t2_meta)

## 3.2. Merge GBM, Metastasis and Resize

In [13]:
def gbm_meta_merge(gbm, meta):
    x = np.concatenate((gbm, meta), axis=0)
    y = np.concatenate((np.zeros(len(gbm)), np.ones(len(meta))), axis=0)
    return x, y

### 3.2.1. Mask

In [14]:
def mask_resize(data, size=(224, 224)):
    mask_resized = []
    for x in data:
        resize = cv2.resize(x, size, interpolation=cv2.INTER_NEAREST_EXACT)
        mask_resized.append(resize)
    return np.array(mask_resized)

In [161]:
train_mask, _ = gbm_meta_merge(x_train_gbm_mask, x_train_meta_mask)
valid_mask, _ = gbm_meta_merge(x_val_gbm_mask, x_val_meta_mask)
test_mask, _ = gbm_meta_merge(x_test_gbm_mask, x_test_meta_mask)
ext_mask, _ = gbm_meta_merge(x_ext_gbm_mask, x_ext_meta_mask)

train_mask = mask_resize(train_mask)
valid_mask = mask_resize(valid_mask)
test_mask = mask_resize(test_mask)
ext_mask = mask_resize(ext_mask)

In [162]:
train_mask.shape, valid_mask.shape, test_mask.shape, ext_mask.shape

((1785, 224, 224), (205, 224, 224), (355, 224, 224), (715, 224, 224))

In [117]:
#save mask file 
data_dir = os.path.join(os.getcwd(), 'DataProcessed/final_mask')

with open(os.path.join(data_dir, 'train_mask.pickle'), 'wb') as f:
    pickle.dump(train_mask, f)
with open(os.path.join(data_dir, 'valid_mask.pickle'), 'wb') as f:
    pickle.dump(valid_mask, f)
with open(os.path.join(data_dir, 'test_mask.pickle'), 'wb') as f:
    pickle.dump(test_mask, f)
with open(os.path.join(data_dir, 'ext_mask.pickle'), 'wb') as f:
    pickle.dump(ext_mask, f)

### 3.2.2. Train, validation, test data

In [15]:
def data_resize(data, size=(224, 224)):
    data_resized = []
    for array in data:
        img = Image.fromarray(array.astype(np.float32))
        img = img.resize(size)
        img = np.expand_dims(np.array(img), -1)
        data_resized.append(img)
    
    return np.array(data_resized)

In [164]:
# Train data
train_ct1, y_train = gbm_meta_merge(x_train_gbm_ct1, x_train_meta_ct1)
train_t2, _ = gbm_meta_merge(x_train_gbm_t2, x_train_meta_t2)

train_ct1 = data_resize(train_ct1)
train_t2 = data_resize(train_t2)

In [165]:
# Validation data
valid_ct1, y_valid = gbm_meta_merge(x_val_gbm_ct1, x_val_meta_ct1)
valid_t2, _ = gbm_meta_merge(x_val_gbm_t2, x_val_meta_t2)

valid_ct1 = data_resize(valid_ct1)
valid_t2 = data_resize(valid_t2)

In [166]:
# Internal test data
test_ct1, y_test = gbm_meta_merge(x_test_gbm_ct1, x_test_meta_ct1)
test_t2, _ = gbm_meta_merge(x_test_gbm_t2, x_test_meta_t2)

test_ct1 = data_resize(test_ct1)
test_t2 = data_resize(test_t2)

In [167]:
# External test data
ext_ct1, y_ext = gbm_meta_merge(x_ext_gbm_ct1, x_ext_meta_ct1)
ext_t2, _ = gbm_meta_merge(x_ext_gbm_t2, x_ext_meta_t2)

ext_ct1 = data_resize(ext_ct1)
ext_t2 = data_resize(ext_t2)

In [168]:
train_ct1.shape, train_t2.shape, y_train.shape

((1785, 224, 224, 1), (1785, 224, 224, 1), (1785,))

In [169]:
valid_ct1.shape, valid_t2.shape, y_valid.shape

((205, 224, 224, 1), (205, 224, 224, 1), (205,))

In [170]:
test_ct1.shape, test_t2.shape, y_test.shape

((355, 224, 224, 1), (355, 224, 224, 1), (355,))

In [171]:
ext_ct1.shape, ext_t2.shape, y_ext.shape

((715, 224, 224, 1), (715, 224, 224, 1), (715,))

In [ ]:
# save y data
data_dir = os.path.join(os.getcwd(), 'DataProcessed/final_twoT2')

with open(os.path.join(data_dir, 'y_train.pickle'), 'wb') as f:
    pickle.dump(y_train, f)
with open(os.path.join(data_dir, 'y_valid.pickle'), 'wb') as f:
    pickle.dump(y_valid, f)
with open(os.path.join(data_dir, 'y_test.pickle'), 'wb') as f:
    pickle.dump(y_test, f)
with open(os.path.join(data_dir, 'y_ext.pickle'), 'wb') as f:
    pickle.dump(y_ext, f)

## 3.3. Data Rescale

In [172]:
# Each modality rescale
ct1_scaler = MinMaxScaler()
t2_scaler = MinMaxScaler()

In [16]:
def scaler_transform(data, scaler): #need to fit on train first
    d1, d2, d3, d4 = data.shape
    data = data.reshape(-1, d4)
    data = scaler.transform(data).reshape(d1, d2, d3, d4)
    return data

### 3.3.1. CT1

In [179]:
# fit & transform on train data (ct1)
d1, d2, d3, d4 = train_ct1.shape
train_ct1 = train_ct1.reshape(-1, d4)
ct1_scaler.fit(train_ct1)
train_ct1 = ct1_scaler.transform(train_ct1).reshape(d1, d2, d3, d4)

In [180]:
valid_ct1 = scaler_transform(valid_ct1, ct1_scaler)
test_ct1 = scaler_transform(test_ct1, ct1_scaler)
ext_ct1 = scaler_transform(ext_ct1, ct1_scaler)

In [ ]:
# visualizing extracted data
n=0
f, axarr = plt.subplots(10, 10, sharex=True, sharey=True, figsize=(20, 20))
for i in range(10):
    for j in range(10):
        # put dataset you want to see
        axarr[i, j].imshow(train_ct1[n+10*i+j][:,:,4], cmap='gray')
        axarr[i, j].axis('off')
        axarr[i, j].set_title(n+10*i+j)
plt.show()

In [ ]:
print('CT1_Train Min-Max: ', train_ct1.min(), train_ct1.max())
print('CT1_Valid Min-Max: ', valid_ct1.min(), valid_ct1.max())
print('CT1_Test Min-Max: ', test_ct1.min(), test_ct1.max())
print('CT1_Ext Min-Max: ', ext_ct1.min(), ext_ct1.max())

### 3.3.2. T2

In [182]:
# fit & transform on train data (t2)
d1, d2, d3, d4 = train_t2.shape
train_t2 = train_t2.reshape(-1, d4)
t2_scaler.fit(train_t2)
train_t2 = t2_scaler.transform(train_t2).reshape(d1, d2, d3, d4)

In [183]:
valid_t2 = scaler_transform(valid_t2, t2_scaler)
test_t2 = scaler_transform(test_t2, t2_scaler)
ext_t2 = scaler_transform(ext_t2, t2_scaler)

In [ ]:
# visualizing extracted data
n=0
f, axarr = plt.subplots(10, 10, sharex=True, sharey=True, figsize=(20, 20))
for i in range(10):
    for j in range(10):
        # put dataset you want to see
        axarr[i, j].imshow(train_t2[n+10*i+j][:,:,4], cmap='gray')
        axarr[i, j].axis('off')
        axarr[i, j].set_title(n+10*i+j)
plt.show()

In [ ]:
print('T2_Train Min-Max: ', train_t2.min(), train_t2.max())
print('T2_Valid Min-Max: ', valid_t2.min(), valid_t2.max())
print('T2_Test Min-Max: ', test_t2.min(), test_t2.max())
print('T2_Ext Min-Max: ', ext_t2.min(), ext_t2.max())

### 3.3.3. Data (CT1, T2) Concat

In [17]:
def concat_ct1_t2(ct1, t2):
    
    final_data = []
    
    for i in range(len(ct1)):
        concat = np.concatenate((t2[i], t2[i], ct1[i]), axis=2)
        final_data.append(concat)
        
    return np.array(final_data)

In [188]:
x_train = concat_ct1_t2(train_ct1, train_t2)
x_valid = concat_ct1_t2(valid_ct1, valid_t2)
x_test = concat_ct1_t2(test_ct1, test_t2)
x_ext = concat_ct1_t2(ext_ct1, ext_t2)

In [8]:
data_dir = os.path.join(os.getcwd(), 'DataProcessed/final_twoT2')

with open(os.path.join(data_dir, 'x_train.pickle'), 'wb') as f:
    pickle.dump(x_train, f)
with open(os.path.join(data_dir, 'x_valid.pickle'), 'wb') as f:
    pickle.dump(x_valid, f)
with open(os.path.join(data_dir, 'x_test.pickle'), 'wb') as f:
    pickle.dump(x_test, f)
with open(os.path.join(data_dir, 'x_ext.pickle'), 'wb') as f:
    pickle.dump(x_ext, f)

In [ ]:
# visualizing extracted data
n=0
f, axarr = plt.subplots(10, 10, sharex=True, sharey=True, figsize=(20, 20))
for i in range(10):
    for j in range(10):
        # put dataset you want to see
        axarr[i, j].imshow(x_train[n+10*i+j][:,:,4], cmap='gray')
        axarr[i, j].axis('off')
        axarr[i, j].set_title(n+10*i+j)
plt.show()

# 4. OOD Data
preprocessing OOD (meningioma) data

## 4.1. Data paths

In [18]:
data_dir= '../'

ewha_dir = os.path.join(data_dir, 'ewha/03_Crop_n_padded')
sev_dir = os.path.join(data_dir, 'sev')

In [19]:
# mask
ewha_dir_mask = glob(os.path.join(ewha_dir, '*/*_T1C-label_resized.nii'))
sev_dir_mask = glob(os.path.join(sev_dir, '*/*_T1C-label_resized.nii'))

# T1C
ewha_dir_t1c = glob(os.path.join(ewha_dir, '*/*_T1C_resized.nii'))
sev_dir_t1c = glob(os.path.join(sev_dir, '*/*_T1C_resized.nii'))

# T2
ewha_dir_t2 = glob(os.path.join(ewha_dir, '*/*_T2_resized.nii'))
sev_dir_t2 = glob(os.path.join(sev_dir, '*/*_T2_resized.nii'))

In [20]:
len(glob(os.path.join(ewha_dir, '*'))), len(glob(os.path.join(sev_dir, '*')))

(62, 257)

In [21]:
len(glob(os.path.join(ewha_dir, '*/*.nii'))), len(glob(os.path.join(sev_dir, '*/*.nii')))

(186, 771)

## 4.2. Check shapes

In [ ]:
check_shape_difference(ewha_dir_mask, ewha_dir_t1c, ewha_dir_t2)
check_shape_difference(sev_dir_mask, sev_dir_t1c, sev_dir_t2)

## 4.3. Extracting top 5 slices by tumor segmentation (label) data

In [ ]:
ewha_index = total_index(ewha_dir_mask)
sev_index = total_index(sev_dir_mask)

In [ ]:
ewha_t1c = get_top5slices(ewha_dir_t1c, ewha_index)
sev_t1c = get_top5slices(sev_dir_t1c, sev_index)

ewha_t2 = get_top5slices(ewha_dir_t2, ewha_index)
sev_t2 = get_top5slices(sev_dir_t2, sev_index)

In [32]:
ewha_t1c = TurnInto_2DArray(ewha_t1c)
sev_t1c = TurnInto_2DArray(sev_t1c)

ewha_t2 = TurnInto_2DArray(ewha_t2)
sev_t2 = TurnInto_2DArray(sev_t2)

## 4.4. Extracting label

In [25]:
#create label data
df = pd.read_excel(os.path.join(data_dir, 'ID_Label_Mapping_byCenters.xlsx'))

### 4.4.1. Ewha

In [ ]:
df_e = df.loc[df['center']=='Ewha']
df_e = df_e.reset_index()
df_e = df_e.drop(columns='index')
df_e['fnum'] = '-'

for i in range(len(df_e)):
    df_e['fnum'][i] = df_e['filename'][i].split('_')[0]

In [27]:
y_ewha = []
for i, path in enumerate(ewha_dir_t1c):
    fname = path.split('/')[-2]
    index = df_e.index[df_e['fnum']==fname].tolist()
    for i in range(5):
        y_ewha.append(int(df_e['label'][index]))

### 4.4.2. Severance

In [ ]:
df_s = df.loc[df['center']=='Severance']
df_s = df_s.reset_index()
df_s = df_s.drop(columns='index')
df_s['fnum'] = '-'

for i in range(len(df_s)):
    df_s['fnum'][i] = df_s['filename'][i].split('_')[0]

In [29]:
y_sev = []
for i, path in enumerate(sev_dir_t1c):
    fname = path.split('/')[-2]
    index = df_s.index[df_s['fnum']==fname].tolist()
    for i in range(5):
        y_sev.append(int(df_s['label'][index]))

## 4.5. Data resize

In [33]:
ewha_t1c = data_resize(ewha_t1c)
ewha_t2 = data_resize(ewha_t2)

sev_t1c = data_resize(sev_t1c)
sev_t2 = data_resize(sev_t2)

## 4.6. Data rescale

### 4.6.1. Ewha

In [34]:
#re-run for each hospital data
ct1_scaler = MinMaxScaler()
t2_scaler = MinMaxScaler()

In [35]:
# fit & transform on train data (ct1)
d1, d2, d3, d4 = ewha_t1c.shape
ewha_t1c = ewha_t1c.reshape(-1, d4)
ct1_scaler.fit(ewha_t1c)
ewha_t1c = ct1_scaler.transform(ewha_t1c).reshape(d1, d2, d3, d4)

In [36]:
# fit & transform on train data (t2)
d1, d2, d3, d4 = ewha_t2.shape
ewha_t2 = ewha_t2.reshape(-1, d4)
t2_scaler.fit(ewha_t2)
ewha_t2 = t2_scaler.transform(ewha_t2).reshape(d1, d2, d3, d4)

### 4.6.2. Severance

In [37]:
#re-run for each hospital data
ct1_scaler = MinMaxScaler()
t2_scaler = MinMaxScaler()

In [38]:
# fit & transform on train data (ct1)
d1, d2, d3, d4 = sev_t1c.shape
sev_t1c = sev_t1c.reshape(-1, d4)
ct1_scaler.fit(sev_t1c)
sev_t1c = ct1_scaler.transform(sev_t1c).reshape(d1, d2, d3, d4)

In [39]:
# fit & transform on train data (t2)
d1, d2, d3, d4 = sev_t2.shape
sev_t2 = sev_t2.reshape(-1, d4)
t2_scaler.fit(sev_t2)
sev_t2 = t2_scaler.transform(sev_t2).reshape(d1, d2, d3, d4)

### 4.7. Data Concat

In [40]:
x_ewha = concat_ct1_t2(ewha_t1c, ewha_t2)
x_sev = concat_ct1_t2(sev_t1c, sev_t2)

In [ ]:
with open(os.path.join(), 'wb') as f:
    pickle.dump(x_ewha, f)
    
with open(os.path.join(), 'wb') as f:
    pickle.dump(y_ewha, f)
    
with open(os.path.join(), 'wb') as f:
    pickle.dump(x_sev, f)
    
with open(os.path.join(), 'wb') as f:
    pickle.dump(y_sev, f)

In [ ]:
# visualizing extracted data
n=0
f, axarr = plt.subplots(10, 10, sharex=True, sharey=True, figsize=(20, 20))
for i in range(10):
    for j in range(10):
        # put dataset you want to see
        axarr[i, j].imshow(x_sev[n+50*i+5*j], cmap='gray')
        axarr[i, j].axis('off')
        axarr[i, j].set_title(n+50*i+5*j)
plt.show()

In [47]:
x_ewha.shape, np.array(y_ewha).shape, x_sev.shape, np.array(y_sev).shape

((310, 224, 224, 3), (310,), (1285, 224, 224, 3), (1285,))